In [ ]:
import pyproj

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# http://www.spatialreference.org/ref/epsg/3646/
or_north_plain = pyproj.Proj(init='EPSG:3646', preserve_units=True)

In [ ]:
def load_csv(fname):
    falsies = ['No', 'NO','no', ]
    trusies = ['YES', 'Yes', 'yes']
    df = pd.read_csv(fname,
                     true_values=trusies,
                     false_values=falsies,
                     parse_dates=[0],
                     low_memory=False,
                     )
    return df
    

In [ ]:
def transform_coordinates(df, projection):
    # 0 values in X and Y columns are invalid - replace with NaN so we can fix erronous projections
    df.X.replace(0, np.nan, inplace=True)
    df.Y.replace(0, np.nan, inplace=True)
    X = np.array(df.X)
    Y = np.array(df.Y)
    
    lons, lats = projection(X, Y, inverse=True)
    # Projections that don't make sense return 1e+30 rather than erroring.
    # We will replace these with NaNs
    lons[lons == 1e+30] = np.nan
    lats[lats == 1e+30] = np.nan
    df['lon'] = lons
    df['lat'] = lats
    # Get rid of old columns
    df.drop(columns=['X', 'Y'], inplace=True)
    
    return df

In [ ]:
CSV_FILES = ['old_format_combined.csv', 'new_format_combined.csv']

In [349]:
for fname in CSV_FILES:
    df = load_csv(fname)
    completed = transform_coordinates(df, or_north_plain)
    new_name = '_processed.'.join(fname.split('.'))
    df.to_csv(new_name, index=False)

In [348]:
df.to_csv?